# EDA
1. В наборе данных есть дубликаты. 
2. Нарушена равномерность временных интервалов, т.е. не все значения отстоят друг от друга на 1 час.
3. Надо скорректировать колонку holiday:
праздники проставлены только для полночи, надо скопировать значение праздника на остальные часы данных суток.

## Необходимые импорты и определения

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import warnings
from datetime import datetime

import pandas as pd
import plotly.express as px

warnings.filterwarnings("ignore")

In [3]:
def correct_holidays_in_df(df):
    """ Заполняет записи о праздниках для всех часов праздничных дней."""
    current_day = datetime(2000, 1, 1, 0, 0, 0)
    current_holiday = 'None'
    for i, it in enumerate(df.itertuples()):
        if current_day != it.date_time.date():
            current_day = it.date_time.date()
            current_holiday = it.holiday
        elif current_holiday != 'None':
            df.loc[i, 'holiday'] = current_holiday


def interpolate_data(df):
    """ Интерполирует набор данных с шагом 1 час. """
    # Convert 'date_time' to datetime format
    df['date_time'] = pd.to_datetime(df['date_time'])
    # print(df['date_time'].dtype)

    # Remove duplicate date_time entries
    df_no_duplicates_in_date_time = df.drop_duplicates(subset='date_time')

    # Generate a complete datetime range at 1-hour intervals
    min_date = df_no_duplicates_in_date_time['date_time'].min()
    max_date = df_no_duplicates_in_date_time['date_time'].max()
    completed_range = pd.date_range(
        start=min_date,
        end=max_date, freq='H'
    )

    # Set 'date_time' as index (required for time-based interpolation)
    df_no_duplicates_in_date_time.set_index('date_time', inplace=True)

    # Reindex the DataFrame with the complete datetime range
    df_reindexed = df_no_duplicates_in_date_time.reindex(completed_range)

    # Perform interpolation on the DataFrame based on the new datetime index
    df_interpolated = df_reindexed.interpolate(method='time')

    # Forward fill the 'holiday' column to copy the value from previous entries
    df_interpolated['holiday'] = df_interpolated['holiday']\
        .fillna(method='ffill')

    # Ensure 'traffic_volume' is converted to integer
    df_interpolated['traffic_volume'] = df_interpolated['traffic_volume']\
        .round().astype(int)

    # Reset index
    df_interpolated.reset_index(inplace=True)

    df_interpolated.rename(columns={'index': 'date_time'}, inplace=True)

    correct_holidays_in_df(df_interpolated)

    return df_interpolated

In [4]:
# Select colums
columns = ['date_time', 'holiday', 'temp', 'traffic_volume']

df_history = pd.read_csv(
    r'.\Metro_Interstate_Traffic_Volume.csv',
    usecols=columns
)
df_history.loc[df_history["holiday"].isnull(), "holiday"] = "None"
print(df_history.head)

<bound method NDFrame.head of       holiday    temp            date_time  traffic_volume
0        None  288.28  2012-10-02 09:00:00            5545
1        None  289.36  2012-10-02 10:00:00            4516
2        None  289.58  2012-10-02 11:00:00            4767
3        None  290.13  2012-10-02 12:00:00            5026
4        None  291.14  2012-10-02 13:00:00            4918
...       ...     ...                  ...             ...
48199    None  283.45  2018-09-30 19:00:00            3543
48200    None  282.76  2018-09-30 20:00:00            2781
48201    None  282.73  2018-09-30 21:00:00            2159
48202    None  282.09  2018-09-30 22:00:00            1450
48203    None  282.12  2018-09-30 23:00:00             954

[48204 rows x 4 columns]>


В наборе данных есть дубликаты:

In [5]:
ids = df_history["date_time"]
print(
    df_history[ids.isin(ids[ids.duplicated()])].sort_values("date_time").head
)

<bound method NDFrame.head of       holiday    temp            date_time  traffic_volume
178      None  281.25  2012-10-10 07:00:00            6793
179      None  281.25  2012-10-10 07:00:00            6793
180      None  280.10  2012-10-10 08:00:00            6283
181      None  280.10  2012-10-10 08:00:00            6283
182      None  279.61  2012-10-10 09:00:00            5680
...       ...     ...                  ...             ...
48173    None  280.68  2018-09-29 19:00:00            3818
48192    None  283.48  2018-09-30 14:00:00            4380
48193    None  283.48  2018-09-30 14:00:00            4380
48194    None  283.84  2018-09-30 15:00:00            4302
48195    None  283.84  2018-09-30 15:00:00            4302

[13074 rows x 4 columns]>


Удалим дубликаты и интерполируем временную шкалу, чтобы `date_time` было с интервалом 1 час. Также, обработаем колонку holiday: заполним значение праздника на все часы данных суток.

In [6]:
df_no_duplicates = df_history.drop_duplicates()
# print(df_no_duplicates.head)

df_history = interpolate_data(df_no_duplicates)
print(df_history.head)

<bound method NDFrame.head of                 date_time holiday    temp  traffic_volume
0     2012-10-02 09:00:00    None  288.28            5545
1     2012-10-02 10:00:00    None  289.36            4516
2     2012-10-02 11:00:00    None  289.58            4767
3     2012-10-02 12:00:00    None  290.13            5026
4     2012-10-02 13:00:00    None  291.14            4918
...                   ...     ...     ...             ...
52546 2018-09-30 19:00:00    None  283.45            3543
52547 2018-09-30 20:00:00    None  282.76            2781
52548 2018-09-30 21:00:00    None  282.73            2159
52549 2018-09-30 22:00:00    None  282.09            1450
52550 2018-09-30 23:00:00    None  282.12             954

[52551 rows x 4 columns]>


Построим график, чтобы понять сезонность и выбрать: какой объем данных оставить для тренировки моделей.

In [7]:
# Plot using Plotly
fig = px.line(
    df_history,
    x='date_time',
    y='traffic_volume',
    title='Traffic Volume Over Time'
)
fig.update_xaxes(title='Date Time')
fig.update_yaxes(title='Traffic Volume')

# Show interactive plot
fig.show()

Данные до 2016 года сильно отличаются и один год пропущен. Периодичность визуально угадывается в данных начиная с 2016 года и дальше, поэтому для тренировки оставим только данные с 1 января 2016 года.

In [8]:
df_history = df_history.query("date_time > '2016-01-01'")

In [9]:
df_history.to_csv(
    r'.\Metro_Interstate_Traffic_Volume_interpolated.csv',
    index=False
)

Предобработанный интерполированный набор данных сохранили в файл Metro_Interstate_Traffic_Volume_interpolated.csv.